In [1]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from torch import nn
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
import matplotlib.pyplot as plt
import optuna

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [3]:
torch.manual_seed(42)  # for reproducibility

In [4]:
df1 = pd.read_csv('/home/darshan39/Downloads/fashionmnist/fashion-mnist_train.csv')
df2 = pd.read_csv('/home/darshan39/Downloads/fashionmnist/fashion-mnist_test.csv')
df = pd.concat([df1, df2], ignore_index=True)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70000 entries, 0 to 69999
Columns: 785 entries, label to pixel784
dtypes: int64(785)
memory usage: 419.2 MB


In [6]:
x = df.iloc[:, 1:].values
y = df.iloc[:, 0].values

In [7]:
x_train, x_test, y_train, y_test = train_test_split(
    x, y,
    test_size=0.2,
    random_state=42,
)

In [ ]:
import time
import torch

# Device info
print('Using device:', device)
if torch.cuda.is_available():
    try:
        print('GPU name:', torch.cuda.get_device_name(0))
    except Exception as e:
        print('GPU name lookup failed:', e)

# Create a fresh model instance (matches typical trial settings)
input_size = 784
output_size = 10
num_layers = 2
neurons_per_layer = 64
model = newNN(input_size, output_size, num_layers, neurons_per_layer).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1, weight_decay=1e-4)

# Accurate timing when using CUDA: synchronize before/after measuring
if device.type == 'cuda':
    torch.cuda.synchronize()
start = time.time()

model.train()
total_loss = 0.0
batches = 0
for batch_features, batch_labels in train_loader:
    batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)
    optimizer.zero_grad()
    outputs = model(batch_features)
    loss = criterion(outputs, batch_labels)
    loss.backward()
    optimizer.step()
    total_loss += loss.item()
    batches += 1

if device.type == 'cuda':
    torch.cuda.synchronize()
elapsed = time.time() - start

avg_loss = total_loss / batches if batches else float('nan')
print(f'One epoch finished — elapsed: {elapsed:.3f} s ({elapsed/60:.3f} min), batches: {batches}, avg loss: {avg_loss:.6f}')
from torchvision import transforms
custom_transform = transforms.Compose([
    transforms.Resize((256)),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [26]:
from PIL import Image
import numpy as np
class CustomDataset(Dataset):
    def __init__(self, features, labels, transform):
        self.features = features
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        image = self.features[idx].reshape(28, 28).astype(np.uint8)
        image = np.stack([image]*3, axis=-1) 
        image = Image.fromarray(image)
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(label)

In [27]:
train_dataset = CustomDataset(x_train, y_train, transform=custom_transform)

In [28]:
test_dataset = CustomDataset(x_test, y_test, transform=custom_transform)

In [29]:

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, pin_memory=True)

In [30]:
import torchvision.models as models
vgg16 = models.vgg16(pretrained=True)

/home/darshan39/miniconda3/lib/python3.13/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/darshan39/miniconda3/lib/python3.13/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [31]:
vgg16.classifier

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=1000, bias=True)
)

In [32]:
for param in vgg16.features.parameters():
    param.requires_grad = False

In [33]:
vgg16.classifier = nn.Sequential(
    nn.Linear(25088, 1024),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(1024, 512),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 10)
)

In [34]:
vgg16

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [35]:
vgg16 = vgg16.to(device)

In [36]:
learning_rate = 0.0001
epochs = 10

In [37]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(vgg16.classifier.parameters(), lr=learning_rate)

In [38]:
for epoch in range(epochs):
    total_loss = 0.0
    for bacth_fetures, batch_labels in train_loader:
        batch_fetures, batch_labels = bacth_fetures.to(device), batch_labels.to(device)

        outputs = vgg16(batch_fetures)
        loss = criterion(outputs, batch_labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss = total_loss + loss.item()
    
    epoch_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}, Loss: {epoch_loss:.4f}")

Epoch 1, Loss: 2.2355
Epoch 2, Loss: 2.0763
Epoch 3, Loss: 1.9812
Epoch 4, Loss: 1.9348
Epoch 5, Loss: 1.9006
Epoch 6, Loss: 1.8777
Epoch 7, Loss: 1.8590
Epoch 8, Loss: 1.8416
Epoch 9, Loss: 1.8279
Epoch 10, Loss: 1.8209


In [39]:
vgg16.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for batch_fetures, batch_labels in test_loader:
        batch_fetures, batch_labels = batch_fetures.to(device), batch_labels.to(device)
        outputs = vgg16(batch_fetures)
        _, predicted = torch.max(outputs.data, 1)
        total += batch_labels.size(0)
        correct += (predicted == batch_labels).sum().item()
print(f"Test Accuracy: {100 * correct / total:.2f}%")

In [ ]:
total = 0
correct = 0
with torch.no_grad():
    for batch_fetures, batch_labels in train_loader:
        batch_fetures, batch_labels = batch_fetures.to(device), batch_labels.to(device)
        outputs = vgg16(batch_fetures)
        _, predicted = torch.max(outputs.data, 1)
        total += batch_labels.size(0)
        correct += (predicted == batch_labels).sum().item()
print(f"Train Accuracy: {100 * correct / total:.2f}%")

Train Accuracy: 99.88%
